In [2]:
import os
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, ArrayType, IntegerType, FloatType
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.7"
spark = SparkSession.builder.\
        config('spark.executor.memory', '6g').\
        config('spark.executor.cores', '8').\
        config('spark.driver.memory','6g').\
        config('spark.executor.instances', '30').\
        appName('test10').\
        enableHiveSupport().getOrCreate()

In [3]:
trainPos1= spark.sql('select distinct t1.user_guid,t2.is_ride_card, t2.consume_card_cnt,t2.ev_consume_card_cnt,t2.purchase_cnt, \
                         t2.ev_all_total_cnt,t2.total_card_money,t2.total_cnt \
                         from ods.t_driver_approve t1 inner join bikedw.t_dw_bike_user_label t2  on t1.user_guid=t2.user_guid  \
                  where t1.pt="20190630" and t1.approve_status=1 and substr(t1.create_time,1,10) between "2019-06-01" and "2019-06-30"\
                  and t2.pt="20190601"  and t2.last_ride_time <= 30 and t2.last_ride_time>0 ')

In [ ]:
# trainPos1.shape

In [4]:
trainPos1.createOrReplaceTempView("trainPos1")
trainPos2= spark.sql('select t1.user_guid,t3.age,t1.is_ride_card, t1.consume_card_cnt,t1.ev_consume_card_cnt,t1.purchase_cnt,t1.ev_all_total_cnt,\
                      t1.total_card_money,t1.total_cnt, t2.sum_cal, t2.sum_ride_time, t2.sum_ride_number, t2.sum_credit_score \
                      from trainPos1 t1  join ods.t_bike_user t2  join dw.dw_user_mid_ds t3\
                      on t1.user_guid=t2.guid  where t2.pt="20190601"  \
                      and t1.user_guid=t3.user_guid and t3.pt="20190601" and t3.sex=0 and t3.age between 18 and 70 ') 
# trainPos2s=trainPos2.toPandas()
# print(trainPos2s.shape)  # (56051, 13)

In [5]:
click_train = spark.sql('select userguid as user_guid,count(1) as click from ods.t_bike_yukon_pro_pageview \
     where pt >"20190501" and pt<"20190531" and (pageid="APP_首页_顺风车" or pageid="Alipay_首页_顺风车" or pageid="APP_顺风车_首页" \
     or pageid="APP_顺风车_车主_首页" or pageid="Alipay_顺风车_车主_首页")  group by userguid') 
# click_train.createOrReplaceTempView("click_train")  

In [6]:
trainPos=trainPos2.join(click_train,'user_guid','left')  # (56051, 15)

In [7]:
trainPos.createOrReplaceTempView("trainPos")
trainPos=spark.sql('select * , 1 lable from trainPos')  #(56051, 15)

In [8]:
# trainPoss=trainPos.toPandas()
# print(trainPoss.shape)  # (56051, 13)
# trainPos.shape
trainPos

DataFrame[user_guid: string, age: bigint, is_ride_card: bigint, consume_card_cnt: bigint, ev_consume_card_cnt: bigint, purchase_cnt: bigint, ev_all_total_cnt: bigint, total_card_money: decimal(13,5), total_cnt: bigint, sum_cal: bigint, sum_ride_time: bigint, sum_ride_number: bigint, sum_credit_score: bigint, click: bigint, lable: int]

In [9]:
trainNeg1= spark.sql('select t1.user_guid,t1.is_ride_card, t1.consume_card_cnt,t1.ev_consume_card_cnt,t1.purchase_cnt, \
                    t1.ev_all_total_cnt,t1.total_card_money,t1.total_cnt from (select  *  from bikedw.t_dw_bike_user_label \
                     where pt="20190601" and last_ride_time <= 30 and last_ride_time>0 ) t1 left join  \
 (select distinct user_guid  ,1 s from ods.t_driver_approve where substr(create_time,1,10)<="2019-06-30" and pt="20190630") t2  \
   on t1.user_guid=t2.user_guid  where s is null limit 2000000 ')  #--34819269

In [10]:
trainNeg1.createOrReplaceTempView("trainNeg1")
trainNeg2= spark.sql('select distinct t1.user_guid,t3.age,t1.is_ride_card, t1.consume_card_cnt,t1.ev_consume_card_cnt,t1.purchase_cnt, \
                    t1.ev_all_total_cnt,t1.total_card_money,t1.total_cnt,t2.sum_cal,t2.sum_ride_time,t2.sum_ride_number,t2.sum_credit_score \
                    from  trainNeg1 t1 join ods.t_bike_user t2 join dw.dw_user_mid_ds t3\
                     on t1.user_guid=t2.guid  where t2.pt="20190601"  \
                     and t1.user_guid=t3.user_guid and t3.pt="20190601" and t3.sex=0 and t3.age between 18 and 70 ')

In [11]:
trainNeg=trainNeg2.join(click_train,'user_guid','left')  # 

In [12]:
trainNeg.createOrReplaceTempView("trainNeg")
trainNeg=spark.sql('select * , 0  lable from trainNeg ')  #  负样本抽取 1000000  正样本 56051

In [13]:
# trainNegs=trainNeg.toPandas()
# print(trainNegs.shape)
trainNeg

DataFrame[user_guid: string, age: bigint, is_ride_card: bigint, consume_card_cnt: bigint, ev_consume_card_cnt: bigint, purchase_cnt: bigint, ev_all_total_cnt: bigint, total_card_money: decimal(13,5), total_cnt: bigint, sum_cal: bigint, sum_ride_time: bigint, sum_ride_number: bigint, sum_credit_score: bigint, click: bigint, lable: int]

In [14]:
trainNeg.createOrReplaceTempView("trainNeg")
trainPos.createOrReplaceTempView("trainPos")
train_data=spark.sql('select * from trainNeg union select * from trainPos')

In [15]:
# traindatas=traindata.toPandas()
# print(traindatas.shape)  # (1095771, 14)   5万：100万  1：20
train_data

DataFrame[user_guid: string, age: bigint, is_ride_card: bigint, consume_card_cnt: bigint, ev_consume_card_cnt: bigint, purchase_cnt: bigint, ev_all_total_cnt: bigint, total_card_money: decimal(13,5), total_cnt: bigint, sum_cal: bigint, sum_ride_time: bigint, sum_ride_number: bigint, sum_credit_score: bigint, click: bigint, lable: int]

In [16]:
testPos1= spark.sql('select distinct t1.user_guid,t2.is_ride_card, t2.consume_card_cnt,t2.ev_consume_card_cnt,t2.purchase_cnt, \
                         t2.ev_all_total_cnt,t2.total_card_money,t2.total_cnt \
                         from ods.t_driver_approve t1 inner join bikedw.t_dw_bike_user_label t2  on t1.user_guid=t2.user_guid  \
                  where t1.pt="20190731" and t1.approve_status=1 and substr(t1.create_time,1,10) between "2019-07-01" and "2019-07-31"\
                  and t2.pt="20190701"  and t2.last_ride_time <= 30 and t2.last_ride_time>0 ')

In [17]:
testPos1.createOrReplaceTempView("testPos1")
testPos2= spark.sql('select t1.user_guid,t3.age,t1.is_ride_card, t1.consume_card_cnt,t1.ev_consume_card_cnt,t1.purchase_cnt,t1.ev_all_total_cnt,\
                      t1.total_card_money,t1.total_cnt, t2.sum_cal, t2.sum_ride_time, t2.sum_ride_number, t2.sum_credit_score \
                      from testPos1 t1  join ods.t_bike_user t2  join dw.dw_user_mid_ds t3\
                      on t1.user_guid=t2.guid  where t2.pt="20190701"  \
                      and t1.user_guid=t3.user_guid and t3.pt="20190701" and t3.sex=0 and t3.age between 18 and 70 ') 

In [18]:
click_test = spark.sql('select userguid as user_guid,count(1) as click from ods.t_bike_yukon_pro_pageview \
     where pt >"20190601" and pt<"20190630" and (pageid="APP_首页_顺风车" or pageid="Alipay_首页_顺风车" or pageid="APP_顺风车_首页" \
     or pageid="APP_顺风车_车主_首页" or pageid="Alipay_顺风车_车主_首页")  group by userguid') 

In [19]:
testPos=testPos2.join(click_test,'user_guid','left')  # (56051, 15)

In [20]:
testPos.createOrReplaceTempView("testPos")
testPos=spark.sql('select * , 1 lable from testPos')  #(56051, 15)

In [21]:
testNeg1= spark.sql('select t1.user_guid,t1.is_ride_card, t1.consume_card_cnt,t1.ev_consume_card_cnt,t1.purchase_cnt, \
                    t1.ev_all_total_cnt,t1.total_card_money,t1.total_cnt from (select  *  from bikedw.t_dw_bike_user_label \
                     where pt="20190701" and last_ride_time <= 30 and last_ride_time>0 ) t1 left join  \
 (select distinct user_guid  ,1 s from ods.t_driver_approve where substr(create_time,1,10)<="2019-07-31" and pt="20190731") t2  \
   on t1.user_guid=t2.user_guid  where s is null limit 10000000 ')  #--34819269

In [22]:
testNeg1.createOrReplaceTempView("testNeg1")
testNeg2= spark.sql('select distinct t1.user_guid,t3.age,t1.is_ride_card, t1.consume_card_cnt,t1.ev_consume_card_cnt,t1.purchase_cnt, \
                    t1.ev_all_total_cnt,t1.total_card_money,t1.total_cnt,t2.sum_cal,t2.sum_ride_time,t2.sum_ride_number,t2.sum_credit_score \
                    from  testNeg1 t1 join ods.t_bike_user t2 join dw.dw_user_mid_ds t3\
                     on t1.user_guid=t2.guid  where t2.pt="20190701"  \
                     and t1.user_guid=t3.user_guid and t3.pt="20190701" and t3.sex=0 and t3.age between 18 and 70 ')

In [23]:
testNeg=testNeg2.join(click_test,'user_guid','left')  # 

In [24]:
testNeg.createOrReplaceTempView("testNeg")
testNeg=spark.sql('select * , 0  lable from testNeg')  #  

In [25]:
testNeg.createOrReplaceTempView("testNeg")
testPos.createOrReplaceTempView("testPos")
test_data=spark.sql('select * from testNeg union select * from testPos')

In [26]:
test_data

DataFrame[user_guid: string, age: bigint, is_ride_card: bigint, consume_card_cnt: bigint, ev_consume_card_cnt: bigint, purchase_cnt: bigint, ev_all_total_cnt: bigint, total_card_money: decimal(13,5), total_cnt: bigint, sum_cal: bigint, sum_ride_time: bigint, sum_ride_number: bigint, sum_credit_score: bigint, click: bigint, lable: int]

In [ ]:
# testdatas=testdata.toPandas()    
# print(testdatas.shape)  #       5 万：500万  1:100

In [27]:
def age(x):
    if   18 <= x < 25:    y = 0
    elif 25 <= x < 35:    y = 1
    elif 35 <= x < 45:    y = 2
    elif 45 <= x < 55:    y = 3
    elif 55 <= x < 65:    y = 4
    else:  y = 5
    return y

def cal(x):
    if   0 <= x < 500:          y = 0
    elif 500 <= x < 2000:       y = 1
    elif 2000 <= x < 5000:      y = 2
    elif 5000 <= x < 20000:     y = 3
    elif 20000 <= x < 50000:    y = 4
    else:y = 5
    return y

def sum_ride_time(x):
    if   x == 0:             y = 0
    elif 0 < x < 100:        y = 1
    elif 100 <= x < 400:     y = 2
    elif 400 <= x < 600:     y = 3
    elif 600 <= x < 1000:    y = 4
    else:y = 5
    return y

def sum_ride_number(x):
    if   x == 0:             y = 0
    elif 0 < x < 10:        y = 1
    elif 10 <= x < 50:     y = 2
    elif 50 <= x < 100:     y = 3
    else:y = 4
    return y

def sum_credit_score(x): #信用积分
    if   x == 0:             y = 0
    elif 0 < x < 120:        y = 1
    elif 120 <= x < 300:     y = 2
    else:y = 3
    return y
def consume_card_cnt(x): #已消费的单车骑行券数量
    if   x == 0:             y = 0
    elif 0 < x < 3:          y = 1
    elif 3 <= x <= 5:        y = 2
    else:y = 3
    return y
def ev_consume_card_cnt(x): #已消费的助力车骑行券数量
    if   x == 0:             y = 0
    elif 0 < x < 3:          y = 1
    else:y = 2
    return y
def purchase_cnt(x): #购卡次数
    if   x == 0:             y = 0
    elif 0 < x < 3:          y = 1
    else:y = 2
    return y
def ev_all_total_cnt(x): #助力车骑行总次数
    if   x == 0:             y = 0
    elif 0 < x < 5:          y = 1
    else:y = 2
    return y
def total_card_money(x): #购卡总金额
    if   x == 0:            y = 0
    elif 0 < x < 10:        y = 1
    elif 10 <= x < 30:      y = 2
    elif 30 <= x < 50:      y = 3
    else:y = 4
    return y
def total_cnt(x): #打车总次数
    if   x == 0:             y = 0
    elif 0 < x < 5:          y = 1
    else:y = 2
    return y
def click (x): # 顺风车页面点击次数
    if   x == 0:             y = 0
    elif 0 < x <= 10:        y = 1
    elif 10 < x <= 50:       y = 2
    elif 50 < x <= 100:      y = 3
    else:y = 4
    return y

train_data = train_data.toPandas()
print("train_data.shape",train_data.shape)
col1, row = train_data.shape
train_data_x = []
train_data_y = []

for i in range(col1):
    train_data_one = []
    train_data_one.append(age(train_data["age"][i]))
    train_data_one.append(cal(train_data["sum_cal"][i]))
    train_data_one.append(sum_ride_time(train_data["sum_ride_time"][i]))
    train_data_one.append(sum_ride_number(train_data["sum_ride_number"][i]))
    train_data_one.append(sum_credit_score(train_data["sum_credit_score"][i]))
    train_data_one.append(train_data["is_ride_card"][i])
    train_data_one.append(train_data["consume_card_cnt"][i])
    train_data_one.append(train_data["ev_consume_card_cnt"][i])
    train_data_one.append(train_data["purchase_cnt"][i])
    train_data_one.append(ev_all_total_cnt(train_data["ev_all_total_cnt"][i]))
    train_data_one.append(total_card_money(train_data["total_card_money"][i]))
    train_data_one.append(train_data["total_cnt"][i])
    train_data_one.append(click(train_data["click"][i]))
    train_data_y.append(train_data["lable"][i])
    train_data_x.append(train_data_one)
    
print("train save over")

test_data = test_data.toPandas()
print("test_data.shape",test_data.shape)
col2, row = test_data.shape
test_data_x = []
test_data_y = []

for i in range(col2):
    test_data_one = []
    test_data_one.append(age(test_data["age"][i]))
    test_data_one.append(cal(test_data["sum_cal"][i]))
    test_data_one.append(sum_ride_time(test_data["sum_ride_time"][i]))
    test_data_one.append(sum_ride_number(test_data["sum_ride_number"][i]))
    test_data_one.append(sum_credit_score(test_data["sum_credit_score"][i]))
    test_data_one.append(test_data["is_ride_card"][i])
    test_data_one.append(test_data["consume_card_cnt"][i])
    test_data_one.append(test_data["ev_consume_card_cnt"][i])
    test_data_one.append(test_data["purchase_cnt"][i])
    test_data_one.append(ev_all_total_cnt(test_data["ev_all_total_cnt"][i]))
    test_data_one.append(total_card_money(test_data["total_card_money"][i]))
    test_data_one.append(test_data["total_cnt"][i])
    test_data_one.append(click(test_data["click"][i]))
    test_data_y.append(test_data["lable"][i])
    test_data_x.append(test_data_one)

print("test save over")

train_data.shape (1095859, 15)
train save over
test_data.shape (5354065, 15)
test save over


In [29]:
import pandas as pd
train1_data = pd.DataFrame(train_data_x, columns=['age', 'sum_cal', 'sum_ride_time', 'sum_ride_number', 'sum_credit_score',
                                           'is_ride_card', 'consume_card_cnt', 'ev_consume_card_cnt', 'purchase_cnt',
                                           'ev_all_total_cnt', 'total_card_money', 'total_cnt', 'click'])
test1_data = pd.DataFrame(test_data_x, columns=['age', 'sum_cal', 'sum_ride_time', 'sum_ride_number', 'sum_credit_score',
                                           'is_ride_card', 'consume_card_cnt', 'ev_consume_card_cnt', 'purchase_cnt',
                                           'ev_all_total_cnt', 'total_card_money', 'total_cnt', 'click'])

print(train1_data.shape)
print(test1_data.shape)

(1095859, 13)
(5354065, 13)


In [30]:
age = pd.get_dummies(test1_data["age"], prefix='age')
cal = pd.get_dummies(test1_data["sum_cal"], prefix='cal')
test2 = age.join(cal)
ride_time = pd.get_dummies(test1_data["sum_ride_time"], prefix="ride_time")
test2 = test2.join(ride_time)
ride_number = pd.get_dummies(test1_data["sum_ride_number"], prefix="sum_ride_number")
test2 = test2.join(ride_number)
credit_score = pd.get_dummies(test1_data["sum_credit_score"], prefix="sum_credit_score")
test2 = test2.join(credit_score)
test2 = test2.join(test1_data["is_ride_card"])
test2 = test2.join(test1_data["consume_card_cnt"])
test2 = test2.join(test1_data["ev_consume_card_cnt"])
test2 = test2.join(test1_data["purchase_cnt"])
ev_cnt = pd.get_dummies(test1_data["ev_all_total_cnt"], prefix="ev_cnt")
test2 = test2.join(ev_cnt)
card_money = pd.get_dummies(test1_data["total_card_money"], prefix="card_money")
test2 = test2.join(card_money)
test2 = test2.join(test1_data["total_cnt"])
click = pd.get_dummies(test1_data["click"], prefix="click")
test2 = test2.join(click)
test2 = pd.DataFrame(test2, index=None, columns=None)
print("test2 save over")

age = pd.get_dummies(train1_data["age"], prefix='age')
cal = pd.get_dummies(train1_data["sum_cal"], prefix='cal')
train2 = age.join(cal)
ride_time = pd.get_dummies(train1_data["sum_ride_time"], prefix="ride_time")
train2 = train2.join(ride_time)
ride_number = pd.get_dummies(train1_data["sum_ride_number"], prefix="sum_ride_number")
train2 = train2.join(ride_number)
credit_score = pd.get_dummies(train1_data["sum_credit_score"], prefix="sum_credit_score")
train2 = train2.join(credit_score)
train2 = train2.join(train1_data["is_ride_card"])
train2 = train2.join(train1_data["consume_card_cnt"])
train2 = train2.join(train1_data["ev_consume_card_cnt"])
train2 = train2.join(train1_data["purchase_cnt"])
ev_cnt = pd.get_dummies(train1_data["ev_all_total_cnt"], prefix="ev_cnt")
train2 = train2.join(ev_cnt)
card_money = pd.get_dummies(train1_data["total_card_money"], prefix="card_money")
train2 = train2.join(card_money)
train2 = train2.join(train1_data["total_cnt"])
click = pd.get_dummies(train1_data["click"], prefix="click")
train2 = train2.join(click)
train2 = pd.DataFrame(train2, index=None, columns=None)
print("train2 save over")

test2 save over
train2 save over


In [31]:
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_score, accuracy_score, recall_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import auc
from sklearn import metrics
from xgboost import plot_importance


def sta(res_0, a, b):
    q1 = res_0[(res_0['预测为1的概率'] >= a) & (res_0['预测为1的概率'] < b)]
    mat_0 = confusion_matrix(q1['label'], q1['预测结果'], labels=[1, 0])

    new = pd.DataFrame(mat_0.reshape(1, 4), columns=['预测转化_实际转化', '预测未转化_实际转化', '预测转化_实际未转化', '预测未转化_实际未转化'],
                       index=['预测概率{0}-{1}'.format(a, b)])
    new['准确率'] = metrics.accuracy_score(q1['label'], q1['预测结果'])
    metrics.accuracy_score(q1['label'], q1['预测结果'])
    return new

def main():
   
    print("---------begin---------")
    model = LogisticRegression()
    model.fit(train2, train_data_y)

    print("---------train over---------")
    print("w: ", model.coef_)
#     w = model.coef_
#     save = pd.DataFrame(w)
#     save.to_csv(r'D:\work\司机挖掘\w.csv', index=False, header=False)
    print("b: ", model.intercept_)
    precision = model.predict(test2)    
    target = np.array(test_data_y)
    print('精确率:{}'.format(precision_score(precision, target)))
    print('准确率:{}'.format(accuracy_score(precision, target)))
    print('召回率:{}'.format(recall_score(precision, target)))
    print('f1:{}'.format(f1_score(precision, target)))
    pro = model.predict_proba(test2)  
    print('auc:{}'.format(roc_auc_score(target, pro[:, 1])))


    res_3 = pd.concat(
        [pd.DataFrame(target, columns=['label']), pd.DataFrame(precision, columns=['预测结果']), pd.DataFrame(pro, columns=['预测为0的概率', '预测为1的概率'])], axis=1)
    df3 = sta(res_3, 0.9, 1)
    for a, b in [[0.8, 0.9], [0.7, 0.8], [0.6, 0.7], [0.5, 0.6], [0.4, 0.5], [0.3, 0.4], [0.2, 0.3], [0.1, 0.2], [0, 0.1]]:
        df_new = pd.concat([df3, sta(res_3, a, b)])
        df3 = df_new
    print(df_new)

if __name__ == '__main__':
    main()


---------begin---------
---------train over---------
w:  [[-1.37906046e-01  1.06827538e+00  8.01155604e-01  8.85555323e-02
  -7.67326221e-01 -2.11414013e+00 -5.17601372e-02 -4.89748965e-02
  -6.82801825e-02 -7.41293790e-02 -2.55041932e-01 -5.63199370e-01
  -3.42325209e-01  2.68028415e-03 -5.21959175e-02 -1.38448654e-01
  -2.02976304e-01 -3.28120097e-01 -3.38884188e-01 -2.85921546e-01
  -1.86186613e-01 -1.60095474e-01 -9.02980748e-02 -5.59189808e-01
  -3.74534315e-01 -1.21381817e-01 -6.27993348e-03 -1.37148292e-01
   2.71376242e-03 -6.13027163e-03  5.10255698e-02 -4.24432694e-01
  -2.92015711e-01 -3.44937467e-01  4.32133147e-02 -4.19469517e-02
  -1.05889833e-01 -3.24303629e-01 -6.32458773e-01  2.07956818e-04
  -5.78730625e-01  5.81425969e-01  8.04926659e-01 -1.86900788e+00]]
b:  [-1.0613859]
精确率:0.0006397811507236145
准确率:0.9914847877267086
召回率:0.09034267912772585
f1:0.0012705645249622116
auc:0.7286229715599447


ValueError: At least one label specified must be in y_true